1. 导入库与配置

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

# 设置 Scanpy 的绘图参数
sc.settings.verbosity = 3             # 显示日志
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

results_file = 'pbmc3k_results.h5ad'  # 结果保存路径

/tmp/ipykernel_10204/3682314905.py:8: FutureWarning: Use `scanpy.set_figure_params` instead
  sc.settings.set_figure_params(dpi=80, facecolor='white')


2. 下载并读取数据
Scanpy 内置了下载函数，非常方便：

In [12]:
# 下载 3k PBMC 数据（来自 10x Genomics）
# 修改为官方标准函数
adata = sc.datasets.pbmc3k() 
adata.var_names_make_unique()

In [16]:
adata.obs.head() # 查看结果

,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,n_genes
index,,,,,
AAACATACAACCAC-1,781,2421.0,73.0,3.015283,781
AAACATTGAGCTAC-1,1352,4903.0,186.0,3.793596,1352
AAACATTGATCAGC-1,1131,3149.0,28.0,0.889171,1131
AAACCGTGCTTCCG-1,960,2639.0,46.0,1.743085,960
AAACCGTGTATGCG-1,522,981.0,12.0,1.223242,522


3. 预处理与质控 (QC)

In [17]:
# 计算线粒体基因比例， pp 是 Pre-Processing（预处理）的缩写
# sc.pp (Pre-processing)：数据清洗、标准化、质控、去噪。
# sc.tl (Tools)：更高级的分析，如聚类、降维（UMAP）、轨迹追踪。
# sc.pl (Plotting)：绘图和可视化。
# percent_top=None含义：是否计算前 N 个高表达基因所占的比例。作用：通常设为 None 或 [50, 100, 200, 500]。
# 在单细胞中，如果前 50 个基因就占了总表达量的 80%，说明这个细胞的多样性极低，可能是文库质量问题。
# log1p=False含义：是否对计算出的指标进行 $log(x+1)$ 处理。作用：这里设为 False，我们通常直接看原始比例和计数。
# inplace=True含义：是否直接修改原对象。作用：设为 True，意味着该函数没有返回值，而是直接更新你的 adata 变量。
# 这能节省内存（这对你保持 C 盘空间也有好处）。

adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata.obs.head() # 查看结果，你会发现多了几列：
# n_genes_by_counts：每个细胞检测到了多少个基因（基因丰度）。
# total_counts：每个细胞的总 UMI 计数（测序深度）。
# pct_counts_mt：最重要的指标——线粒体基因占比。


/tmp/ipykernel_10204/2406383915.py:11: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['mt'] = adata.var_names.str.startswith('MT-')


,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,n_genes
index,,,,,
AAACATACAACCAC-1,779,2419.0,73.0,3.017776,781
AAACATTGAGCTAC-1,1352,4903.0,186.0,3.793596,1352
AAACATTGATCAGC-1,1129,3147.0,28.0,0.889736,1131
AAACCGTGCTTCCG-1,960,2639.0,46.0,1.743085,960
AAACCGTGTATGCG-1,521,980.0,12.0,1.224490,522


In [20]:

# 过滤细胞
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata = adata[adata.obs.pct_counts_mt < 5, :] # 过滤线粒体占比高的细胞
adata.obs.head(50)


/home/zhen/miniforge3/envs/scanpy_env/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:161: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_genes"] = number


,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,n_genes
index,,,,,
AAACATACAACCAC-1,779,2419.0,73.0,3.017776,779
AAACATTGAGCTAC-1,1352,4903.0,186.0,3.793596,1352
AAACATTGATCAGC-1,1129,3147.0,28.0,0.889736,1129
AAACCGTGCTTCCG-1,960,2639.0,46.0,1.743085,960
AAACCGTGTATGCG-1,521,980.0,12.0,1.224490,521
AAACGCACTGGTAC-1,781,2163.0,36.0,1.664355,781
AAACGCTGACCAGT-1,782,2175.0,83.0,3.816092,782
AAACGCTGGTTCTT-1,790,2260.0,70.0,3.097345,790
AAACGCTGTAGCCA-1,532,1275.0,15.0,1.176471,532


In [ ]:
# 标准化与高变基因选择
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata.obs.head()